# REAME
- [sklearn.datasets.load_breast_cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer)
 を使い、GroupKFold を行う
- EDAは省略


In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%precision 3
%matplotlib inline

# 小さなデータでGroupKFoldの動きを理解する

In [2]:
"""
学生の身長と運動試験の得点のデータを考えます。
ポイントは、学生ごとに所属している部活が違う点です(テニス、サッカー、野球)。
"""
df = pd.DataFrame()
df['height'] = [170, 171, 167, 153, 158, 192]
df['score'] = [34, 29, 55, 21, 19, 50]
df['club'] = ['tennis', 'soccer', 'soccer', 'baseball', 'tennis', 'baseball']
df

,height,score,club
0,170,34,tennis
1,171,29,soccer
2,167,55,soccer
3,153,21,baseball
4,158,19,tennis
5,192,50,baseball


In [3]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=3)

In [4]:
# groups にグループを分ける情報を渡すと、
# 同じグループのデータは 訓練 か 検証 のどちらか固まります
for train_index, valid_index in gkf.split(df, groups=df['club']):
    print(train_index, valid_index)

[1 2 3 5] [0 4]
[0 3 4 5] [1 2]
[0 1 2 4] [3 5]


# cancerデータでやってみる
- 同一の患者のデータが複数あったと仮定しましょう
- 訓練と検証に同じ患者がいると予測が簡単になるので、固まらないようにしたい

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

# 実験なので適当にデータを小さくする
N = 200
X = cancer.data[:N, :2]

# 勝手に患者IDを振ります
X = np.concatenate([X,
                    np.random.randint(0, 5, size=(N, 1))
                    ], axis=1)

y = cancer.target[:N]

X_trainval, X_test, y_trainval, y_test = train_test_split(X,
                                                          y,
                                                          train_size=0.80,
                                                          random_state=0)

In [6]:
# XのみをDFにして確認しておく
df_trainval = pd.DataFrame(X_trainval, columns=['a', 'b', 'patient_id'])
df_test = pd.DataFrame(X_test, columns=['a', 'b', 'patient_id'])

df_trainval.head()

,a,b,patient_id
0,18.450,21.91,4.0
1,9.465,21.01,2.0
2,14.580,21.53,4.0
3,10.510,20.19,3.0
4,17.470,24.68,2.0


## GroupKFoldの場合


In [7]:
gkf = GroupKFold(n_splits=4)


for train_index, valid_index in gkf.split(df_trainval, y_trainval, df_trainval['patient_id']):
    X_train, X_valid = df_trainval.iloc[train_index], df_trainval.iloc[valid_index]
    y_train, y_valid = y_trainval[train_index], y_trainval[valid_index]

    # 同一グループが訓練と検証の両方に登場していないことを確認
    print(X_train['patient_id'].unique(), X_valid['patient_id'].unique())
    # 以降の学習コードは省略

[4. 3. 1. 0.] [2.]
[4. 2. 3. 0.] [1.]
[4. 2. 3. 1.] [0.]
[2. 1. 0.] [4. 3.]



## GroupKFold はシャッフルと乱数シードが使えない問題
- シャッフルが使えない → 偏りが起きてしまうかもしれない
- 乱数シードが使えない → 再現しにくい
- KFoldを使って実装することが多い

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=0)

patient_ids = df_trainval['patient_id']
unique_patient_ids = df_trainval['patient_id'].unique()

for train_group_index, valid_group_index in kf.split(unique_patient_ids):
    train_groups = unique_patient_ids[train_group_index]
    valid_groups = unique_patient_ids[valid_group_index]
    
    # グループごとのIDを抽出していることを確認
    print(train_groups, valid_groups)

    train_index = patient_ids.isin(train_groups)
    valid_index = patient_ids.isin(valid_groups)

    X_train = df_trainval[train_index]
    X_valid = df_trainval[valid_index]
    
    # 同一グループが訓練と検証の両方に登場していないことを確認
    print(X_train['patient_id'].unique(), X_valid['patient_id'].unique())

    # 学習コードは省略(他の交差検証と変わらない)

[2. 1. 0.] [4. 3.]
[2. 1. 0.] [4. 3.]
[4. 3. 1. 0.] [2.]
[4. 3. 1. 0.] [2.]
[4. 2. 3. 0.] [1.]
[4. 2. 3. 0.] [1.]
[4. 2. 3. 1.] [0.]
[4. 2. 3. 1.] [0.]


おわり